### Parameter File Overview:
Parameter files can be TSVs or CSVs. The first line of the file should contain column headings. The following headings are required (in any order): mechanism_id, part_id, param_name, param_val (spaces can be substituted for underscores and headings are not case sensitive). mechanism_id is the name of the Mechanism or the kind of mechanism that will use this parameter, for example "transcription" or  "transcription_mm" for Mechalis-Menten transcription would go in this column. part_id refers to the name of the Component that will use this mechanism, for example "tetR_promoter". param_name refers to the name of the model parameter, for example "ktx", "kb", or "ku". The value of these columns is case sensitive and underscores are different from spaces.

### Parameters inside BioCRNpyler:
Inside of bioCRNpyler, parameters are stored as a dictionary key value pair: (mechanism_name, part_id, param_name) --> param_val. If that particular parameter key cannot be found, the software will default to the following keys: (mechanism_type, part_id, param_name) >> (part_id, param_name) >> (mechanism_name, param_name) >> (mechanism_type, param_name) >>(param_name) and give a warning. This allows for models to be constructed easily using default parameter values. As a note, mechanism_name refers to the .name variable of a Mechanism. mechanism_type refers to the .type variable of a Mechanism. Either of these can be used as a mechanism_id.

### Multiple Parameter Files:
Components and Mixtures can both have one more multiple parameter files. Components use parameters loaded from their file before defaulting to the file supplied to a Mixture. The last file loaded will take precedent and overwrite parameter files which were written earlier.

Below is an example csv with all the parameters for a tetR promoter undergoing Michalis Menten transcription and translation.

In [1]:
from biocrnpyler import *
param_file_name = "Parameters Notebook Param File 1.tsv"

#Create a Regulated Promoter
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD")
tet = Protein("tetR")
components = [reg_rep_assembly, tet]
myMixture = BasicExtract(name="txtl", parameter_file = param_file_name, components=components, parameter_warnings=False)

#Open and print the parameter file
param_file = open(param_file_name)
print("****Parameter File****")
print(param_file.read())
param_file.close()

#Print the parameter dictionary created from the file
print("\n****Loaded Parameters****")
for k in myMixture.parameters:
    print("param[", k, "] = ", myMixture.parameters[k])

****Parameter File****
mechanism_id	part_id	param_name	param_val	comments
transcription_mm	ptet_tetR	kb	10.	extra columns are okay!
transcription_mm	ptet_tetR	ku	.1	These are the parameters for transcription
transcription_mm	ptet_tetR	ktx	1.		
transcription_mm		ktx_leak	1.0	default values might leave out part_id setting a default transcriptional leak rate
	tetR	cooperativity	2	or mechanism_id	meaning that tetR always will default to a cooperativity of 2 for all mechanisms.
		ku_leak	10.	or both! In this case any parameter anmes ku_leak will default to this value.
		kb_leak	10.
one_step_cooperative_binding	ptet_tetR	ku	.1	These are parameters for tetR dimerization and binding with the promoter	
one_step_cooperative_binding	ptet_tetR	kb	.1
translation_mm	BCD	ktl	2.0	These are paraemters for translation
translation_mm	BCD	ku	.25		
translation_mm	BCD	kb	10
rna_degredation_mm		kb	10	These are paraemters for RNA degredation. They will be the same for all RNAs
rna_degredation_mm		ku	.5
rna_de

### Parameter Warnings:
We can see when default parameters are loaded by toggling the 'parameter_warnings' keyword for a Mixture or a Component. By default this is set to None for Mixtures, which means warnings can be toggled by Component. If set to True/False for a Mixture, this will supersede the Component level Toggling. The default setting for Component is parameter_warnings = True. Below if you change the parameter warnings for various Components, the warning messages printed will be different. Note that in Jupyter Notebooks you may need to restart and run all in order for warnings not to be suppressed.

In [2]:
print("Mixture parameter_warnings=None")
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True, parameter_warnings = True)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD", parameter_warnings = True)
components = [reg_rep_assembly, tet]
myMixture = BasicExtract(name="txtl", parameter_file = param_file_name, components=components, parameter_warnings = None)
myCRN = myMixture.compile_crn()
print(myCRN)

Mixture parameter_warnings=None
Species = protein_reporter, complex_rna_reporter_protein_RNAase, complex_protein_2.0xtetR_dna_reporter, rna_reporter, complex_dna_reporter_protein_RNAP, complex_rna_reporter_protein_Ribo, dna_reporter, complex_protein_2.0xtetR_dna_reporter_protein_RNAP, protein_tetR, protein_RNAP, protein_Ribo, protein_RNAase
Reactions = [
	dna_reporter + protein_RNAP <--> complex_dna_reporter_protein_RNAP        massaction: k_f=10.0	k_r=10.0
	complex_dna_reporter_protein_RNAP --> dna_reporter + rna_reporter + protein_RNAP        massaction: k_f=1.0
	2.0*protein_tetR + dna_reporter <--> complex_protein_2.0xtetR_dna_reporter        massaction: k_f=0.1	k_r=0.1
	complex_protein_2.0xtetR_dna_reporter + protein_RNAP <--> complex_protein_2.0xtetR_dna_reporter_protein_RNAP        massaction: k_f=10.0	k_r=0.1
	complex_protein_2.0xtetR_dna_reporter_protein_RNAP --> complex_protein_2.0xtetR_dna_reporter + rna_reporter + protein_RNAP        massaction: k_f=1.0
	rna_reporter + prote

C:\Users\wp_ix\Miniconda3\lib\site-packages\biocrnpyler-0.1-py3.6.egg\biocrnpyler\component.py:8: UserWarning: No Parameter found with param_name=cooperativity and part_id=tetR and mechanism=one_step_cooperative_binding. Parameter found under the key (part_id, param_name)=(tetR, cooperativity)
  pywarn(txt)
C:\Users\wp_ix\Miniconda3\lib\site-packages\biocrnpyler-0.1-py3.6.egg\biocrnpyler\component.py:8: UserWarning: No Parameter found with param_name=ktx_leak and part_id=ptet and mechanism=transcription_mm. Parameter found under the key (mechanism.name, Component.name, param_name)=(transcription_mm, ptet, ktx_leak)
  pywarn(txt)
C:\Users\wp_ix\Miniconda3\lib\site-packages\biocrnpyler-0.1-py3.6.egg\biocrnpyler\component.py:8: UserWarning: No Parameter found with param_name=ku_leak and part_id=ptet and mechanism=transcription_mm. Parameter found under the key param_name=ku_leak
  pywarn(txt)
C:\Users\wp_ix\Miniconda3\lib\site-packages\biocrnpyler-0.1-py3.6.egg\biocrnpyler\component.py:8:

### Setting Parameters at the Component Level
Components can have their own parameter files instead of relying on the parameter files passed into a mixture. Components can also have the mixtures default parameters overwritten with a parameter dictionary. Below, we will create a component which has custom parameters loaded in as a dictionary which works the same as loading them with a file.

In [3]:
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True)

#A custom param dictionary for the regulated assembly
ra_param_dict = {
    ("transcription_mm", "ptet", "ku_leak"):33.33,
     ("transcription_mm", "ptet", "kb_leak"):.33, 
     ("transcription_mm", "ptet", "ktx_leak"):3.33
}
#Pass the param_dict in using the parameters keyword to the Component constructor.
#Use the parameter_file keyword to update the parameters with a file.
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD", parameters = ra_param_dict)
components = [reg_rep_assembly, tet]
myMixture = BasicExtract(name="txtl", parameter_file = param_file_name, components=components)
myCRN = myMixture.compile_crn()
print(myCRN)

Species = protein_reporter, complex_rna_reporter_protein_RNAase, complex_protein_2.0xtetR_dna_reporter, rna_reporter, complex_dna_reporter_protein_RNAP, complex_rna_reporter_protein_Ribo, dna_reporter, complex_protein_2.0xtetR_dna_reporter_protein_RNAP, protein_tetR, protein_RNAP, protein_Ribo, protein_RNAase
Reactions = [
	dna_reporter + protein_RNAP <--> complex_dna_reporter_protein_RNAP        massaction: k_f=0.33	k_r=33.33
	complex_dna_reporter_protein_RNAP --> dna_reporter + rna_reporter + protein_RNAP        massaction: k_f=3.33
	2.0*protein_tetR + dna_reporter <--> complex_protein_2.0xtetR_dna_reporter        massaction: k_f=0.1	k_r=0.1
	complex_protein_2.0xtetR_dna_reporter + protein_RNAP <--> complex_protein_2.0xtetR_dna_reporter_protein_RNAP        massaction: k_f=10.0	k_r=0.1
	complex_protein_2.0xtetR_dna_reporter_protein_RNAP --> complex_protein_2.0xtetR_dna_reporter + rna_reporter + protein_RNAP        massaction: k_f=1.0
	rna_reporter + protein_Ribo <--> complex_rna_repor